# Mathematische Modelle zur Beschreibung von Daten

Wir haben letzte Woche einen Datensatz von handgeschriebenen Ziffern eingelesen. Diese waren im csv-Format gespeichert. Wir haben die handgeschriebenen Ziffern visualisiert und eine Reihe an Statistiken auf den Daten berechnet. 

In einem anderen Notebook, haben wir uns Wahrscheinlichkeitsverteilungen angeschaut. Wir haben zu den Wahrscheinlichkeitsverteilungen Zufallszahlen generiert und diese zusammen mit den Wahrscheinlichkeitsdichtefunktionen visualisiert.

Anhand eines weiteren Datensatzes (https://datahub.io/core/global-temp#data) wollen wir nun die zwei Welten zusammenbringen und ein neues Datenformat (JSON) kennenlernen. Wir wollen diese Daten einlesen und anhand von mathematischen Modellfunktionen beschreiben. Bei dem Datensatz handelt es sich um die monatlichen Temperaturmittelwerte für die gesammte Erde.

---

In [1]:
import numpy as np
import pandas as pd
import scipy, scipy.stats
import matplotlib.pyplot as plt

In [2]:
f = open("monthly_json.json", "r") # öffnet die Datei
file_content = f.read(500) # liest die ersten 100 Zeichen der Datei

FileNotFoundError: [Errno 2] No such file or directory: 'monthly_json.json'

In [ ]:
file_content

Wie wir sehen, handelt es sich immernoch um eine Textdatei. Diese ist jedoch in einem grundsätzlich anderen Format aufgebaut. Jedoch gibt es auch hier bereits fertige Interpreter in pandas:

In [ ]:
data = pd.read_json("monthly_json.json") 

In [ ]:
data.head()

Zunächst will ich die Daten etwas umformatieren, so dass das Datum zum index des Dataframes wird und die verschiedenen Quellen (Source) als Spalten dargestellt werden. Dazu nutze ich die Pivot-Funktion von pandas:

In [ ]:
pivot_df = data.pivot(index='Date', columns='Source', values='Mean')

In [ ]:
pivot_df.head()

Wie ich aus der Beschreibung der Daten weiß, handelt es sich um einen zeitlichen Verlauf der Temperaturmittelwerte. Diesen will ich zunächst einmal visualisieren um ein gefühl dafür zu bekommen.

In [ ]:
pivot_df.plot(figsize=(20, 8),);

Zuletzt bei den Verteilungen haben wir selbst generierte Daten in Zusammenhang mit ihren Wahrscheinlichkeitsdichtefunktionien gebracht. Nun versuche ich dasselbe mit den Temperaturdaten zu denen ich die Verteilung nicht kenne. Dazu visualisiere ich erstmal die Wertebereiche für 'GCAG' und 'GISTEMP' mit Hilfe eines normierten Histogramms.

In [ ]:
plt.figure(1, figsize=(20, 8))

plt.subplot(1,2, 1 )
plt.hist(pivot_df['GCAG'], bins=100, density=True)

plt.subplot(1,2, 2 )
plt.hist(pivot_df['GISTEMP'], bins=100, color='orange')

plt.show()

Und lasse mir ein paar Statistiken dazu ausgeben

In [ ]:
pivot_df.describe()

Auch wenn die Daten nicht ganz normalverteilt aussehen, will ich diese Wahrscheinlichkeitsdichtefunktion über die Werte plotten. Dazu nehme ich die Mittelwert und Standardabweichung aus dem describe() Ergebnis.

In [ ]:
loc_gcag = 0.048797
scale_gcag = 0.326230
lower = scipy.stats.norm.ppf(0.01,loc_gcag,scale_gcag)
upper = scipy.stats.norm.ppf(0.99,loc_gcag,scale_gcag)
x_gcag = scipy.linspace(lower,upper,100)

y_gcag = scipy.stats.norm.pdf(x_gcag,loc_gcag,scale_gcag)

In [ ]:
loc_gistemp = 0.024380
scale_gistemp = 0.343709
lower = scipy.stats.norm.ppf(0.01,loc_gistemp,scale_gistemp)
upper = scipy.stats.norm.ppf(0.99,loc_gistemp,scale_gistemp)
x_gistemp = scipy.linspace(lower,upper,100)

y_gistemp = scipy.stats.norm.pdf(x_gistemp,loc_gistemp,scale_gistemp)

In [ ]:
plt.figure(1, figsize=(20, 8))

plt.subplot(1,2, 1 )
plt.hist(pivot_df['GCAG'], bins=100, density=True)
plt.plot(x_gcag,y_gcag)

plt.subplot(1,2, 2 )
plt.hist(pivot_df['GISTEMP'], bins=100, density=True, color='orange')
plt.plot(x_gistemp,y_gistemp)

plt.show()

Wie unschwer zu erkennen ist, passt die Verteilung nicht genau auf die Daten und ist somit keine besonders gute Beschreibung der Daten.

** AUFGABE: ** Finde eine bessere Möglichkeit die Daten zu beschreiben

In [ ]:
from fitter import Fitter

In [ ]:
f = Fitter(pivot_df, distributions=['gamma', 'rayleigh', 'chi'])
f.fit()
f.summary()

---

In [ ]:
f = Fitter(pivot_df, distributions=None)
f.fit()
f.summary()

In [ ]:
f = Fitter(pivot_df, distributions=['betaprime', 'burr', 'exponnorm', 'genlogistic', 'gumbel_r', 'invweibel', 'moyal', 'vonmises'])
f.fit()
f.summary()